In [24]:
import numpy as np
import pandas as pd

In [25]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [26]:
train_df.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,3600.0


In [27]:
test_df.tail()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
653641,191348,Sunday,6.657211e+10,1,BATH AND SHOWER,1505.0
653642,191348,Sunday,8.818139e+10,1,BATH AND SHOWER,1099.0
653643,191348,Sunday,4.282557e+09,1,MENS WEAR,8220.0
653644,191348,Sunday,8.046919e+10,1,SWIMWEAR/OUTERWEAR,114.0
653645,191348,Sunday,7.871536e+09,1,MENS WEAR,4923.0


원본 유지를 위해서 깊은 복사

In [28]:
def upc_decode(UPC_comp):
    if UPC_comp != 'nan':
        UPC_comp = str(UPC_comp).split('.')[0]
        UPC_company = __upc_to_company(UPC_comp)
    else: UPC_company = -9999
    return UPC_company

In [29]:
def __upc_to_company(UPC_comp):
    upc_company = UPC_comp[:6]
    return upc_company

In [30]:
train_df['Upc'] = train_df['Upc'].apply(upc_decode)

In [31]:
testing_df = train_df[['TripType','Upc','ScanCount']].copy()

일부분만 추출

In [32]:
testing_df.head()

,TripType,Upc,ScanCount
0,999,681131,-1
1,30,605388,1
2,30,741081,1
3,26,223840,2
4,26,200661,2


TripType , FinelineNumber 를 계층적 인덱스로 기준을 잡고 나머지 ScanCount를 더한 것을 value로 하는 데이터프레임 형성

In [33]:
testing_df = testing_df.groupby(by=['TripType','Upc']).sum()

triptype_ls 에는 TripType(3,..,999)의 고유한 값만이 들어간다.

In [34]:
triptype_ls = sorted(list(train_df['TripType'].unique()))
len(triptype_ls)

38

상위 5개의 ScanCount를 뽑아내는 반복문이다. ls_iter list에 하나의 TripType기준으로 상위 5개를 저장해 묶어 저장한다.

In [35]:
ls_iter = []
for i in triptype_ls:
    if not np.isnan(i) :
        ls_iter.append(list(testing_df.loc[i].sort_values(by='ScanCount')['ScanCount'])[-5:])
len(ls_iter)

38

38개의 TripType 을 키로 하고, 상위 5개의 ScanCount값을 value로 하는 dictionary dict_top5

In [36]:
dict_top5 = dict(zip(triptype_ls,ls_iter))
len(dict_top5)

38

In [37]:
ls_fl_testing = []
for i ,j in dict_top5.items():
    for k in j:
        ls_fl_testing.append(list((testing_df.loc[i,:,:][testing_df.loc[:]['ScanCount'] == k].loc[i].index)))
len(ls_fl_testing)

/home/hskimim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


190

In [38]:
ls_fl_testing
ls_fl_testing = [i for l in ls_fl_testing for i in l]

In [39]:
len(ls_fl_testing)

206

In [40]:
ls_fl_testing = list(set(ls_fl_testing))
len(ls_fl_testing)

84

In [41]:
sorted(ls_fl_testing)

['111799',
 '111923',
 '138032',
 '150000',
 '178001',
 '225320',
 '231000',
 '247193',
 '251921',
 '251922',
 '261000',
 '277360',
 '277550',
 '282000',
 '284000',
 '289145',
 '289952',
 '300450',
 '312546',
 '323900',
 '333836',
 '340000',
 '342234',
 '350007',
 '350119',
 '363824',
 '366320',
 '370008',
 '381370',
 '386750',
 '400086',
 '4011',
 '413830',
 '440000',
 '446003',
 '460134',
 '465007',
 '490000',
 '500002',
 '500004',
 '500005',
 '520003',
 '605388',
 '610214',
 '616960',
 '645416',
 '653569',
 '673419',
 '681131',
 '692302',
 '710425',
 '711719',
 '715211',
 '715339',
 '719241',
 '724504',
 '725033',
 '731491',
 '736508',
 '746775',
 '753531',
 '765013',
 '765940',
 '782573',
 '787420',
 '787421',
 '787423',
 '787429',
 '799366',
 '802404',
 '830324',
 '831200',
 '840109',
 '852000',
 '874586',
 '880615',
 '880632',
 '883929',
 '885306',
 '885370',
 '886111',
 '887276',
 '887961',
 'nan']

In [45]:
ls_test = sorted(list(train_df['Upc'].unique()))
len(ls_test)

9097

In [46]:
count = 0
for i in ls_test:
    for j in ls_fl_testing:
        if i == j:
            count+=1

In [47]:
count

84

In [48]:
upc_top5_df = pd.DataFrame()
upc_top5_df['list'] = ls_fl_testing

In [50]:
upc_top5_df.dropna(inplace=True)

In [52]:
upc_top5_df.values

array([['782573'],
       ['753531'],
       ['725033'],
       ['366320'],
       ['852000'],
       ['724504'],
       ['787421'],
       ['465007'],
       ['653569'],
       ['231000'],
       ['284000'],
       ['787429'],
       ['831200'],
       ['645416'],
       ['333836'],
       ['500004'],
       ['874586'],
       ['715211'],
       ['340000'],
       ['886111'],
       ['277550'],
       ['289952'],
       ['736508'],
       ['880615'],
       ['440000'],
       ['605388'],
       ['885306'],
       ['138032'],
       ['765940'],
       ['883929'],
       ['885370'],
       ['731491'],
       ['500002'],
       ['312546'],
       ['300450'],
       ['251921'],
       ['178001'],
       ['840109'],
       ['342234'],
       ['261000'],
       ['765013'],
       ['251922'],
       ['746775'],
       ['400086'],
       ['247193'],
       ['830324'],
       ['616960'],
       ['719241'],
       ['323900'],
       ['787420'],
       ['363824'],
       ['282000'],
       ['401